In [45]:
! pip install lightbgm
#!conda install scikit-learn
import os
import tensorflow as tf	
import numpy as np
import time
from tf_utils import input_fn_from_dataset,input_fn_frame_from_dataset,save_tf_record,prob_positive_class_from_prediction
from get_data import get_videos_from_folder,get_target_from_csv
from utils import save_solution
from keras.layers import Dense, Flatten, Dropout, ZeroPadding3D
from keras.layers.recurrent import LSTM
from keras.models import Sequential, load_model
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import TimeDistributed
from keras.layers.convolutional import (Conv2D, MaxPooling3D, Conv3D,
                                        MaxPooling2D)
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from keras import backend as K
from keras import metrics
from sklearn.utils.validation import check_array
import lightgbm as lgb
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

  Could not find a version that satisfies the requirement lightbgm (from versions: )
No matching distribution found for lightbgm


In [7]:
pframe=40

In [72]:
train_folder = os.path.join("../train/")
test_folder = os.path.join("../test/")

train_target = os.path.join('../train_target.csv')
my_solution_file = os.path.join('../solution.csv')

tf_record_dir = os.path.join('..','tf_records')
os.makedirs(tf_record_dir, exist_ok=True)

tf_record_train = os.path.join(tf_record_dir, 'train' + '.tfrecords')
tf_record_test = os.path.join(tf_record_dir, 'test' + '.tfrecords')
x_train = get_videos_from_folder(train_folder)
y_train = get_target_from_csv(train_target)
X = np.zeros((x_train.shape[0], pframe, 100, 100,1))

for i in range(x_train.shape[0]):
    if x_train[i].shape[0] >= pframe:
        X[i,: pframe,:x_train[i].shape[1],:x_train[i].shape[2],0] = x_train[i][: pframe,:x_train[i].shape[1],:x_train[i].shape[2]]
    else:
        # repeated padding
        frames, height, width = x_train[i].shape[0], x_train[i].shape[1], x_train[i].shape[2]
        pos = 0
        while pos + frames <= pframe:
            X[i,pos:(pos+frames),:height,:width,0] = x_train[i][:frames,:height,:width]
            pos += frames
        if pos < pframe:
            X[i,pos:pframe,:height,:width,0] = x_train[i][:(pframe-pos),:height,:width]

In [41]:
y_train = get_target_from_csv(train_target)

In [ ]:
def cross_validate( split_size=5):
    results = []
    kf = KFold(n_splits=split_size)
    for train_idx, val_idx in kf.split(X, y_train):
        train_x = X[train_idx]
        train_y = y_train[train_idx]
        val_x = X[val_idx]
        val_y = y_train[val_idx]
        His=model.fit(train_x, train_y, batch_size=32, validation_data=(val_x,val_y),verbose=1,callbacks=[tb, early_stopper, csv_logger], epochs=nb_epoch)
        results.append(His,history)
    return results
results=cross_validate(5)

In [73]:
x_test = get_videos_from_folder(test_folder)

In [74]:
X_test = np.zeros((x_test.shape[0],  pframe, 100, 100,1))
for i in range(x_test.shape[0]):
    if x_test[i].shape[0]>= pframe:
        X_test[i,: pframe,:x_test[i].shape[1],:x_test[i].shape[2],0] = x_test[i][: pframe,:x_test[i].shape[1],:x_test[i].shape[2]]
    else:
        # repeated padding
        frames, height, width = x_test[i].shape[0], x_test[i].shape[1], x_test[i].shape[2]
        pos = 0
        while pos + frames <= pframe:
            X_test[i,pos:(pos+frames),:height,:width,0] = x_test[i][:frames,:height,:width]
            pos += frames
        if pos < pframe:
            X_test[i,pos:pframe,:height,:width,0] = x_test[i][:(pframe-pos),:height,:width]

In [10]:
x_cell_edges = np.linspace(0,40,4,dtype=int)
y_cell_edges = np.linspace(0,100,9,dtype=int)
z_cell_edges = np.linspace(0,100,9,dtype=int)

        # histograms
histogram_train = np.zeros((X.shape[0], 3,8,8, 45))
histogram_test = np.zeros((X_test.shape[0], 3,8,8, 45))

In [11]:
for i in range(0, X.shape[0]):
    image_3D = X[i, :, :, :,0]

    for xi in range(0, x_cell_edges.size - 1):
        for yi in range(0, y_cell_edges.size - 1):
            for zi in range(0, z_cell_edges.size - 1):

                        # image block for histogram
                image_block = image_3D[
                                      x_cell_edges[xi]:x_cell_edges[xi+1],
                                      y_cell_edges[yi]:y_cell_edges[yi+1],
                                      z_cell_edges[zi]:z_cell_edges[zi+1]]

                        # histogram
                histogram_train[i, xi, yi, zi, :], bins = \
                            np.histogram(image_block,
                                         bins=np.linspace(0,
                                                          4500,
                                                          45 + 1))

X_new = np.reshape(histogram_train, (X.shape[0], -1))


In [48]:
for i in range(0, X_test.shape[0]):
    image_3D = X_test[i, :, :, :,0]

    for xi in range(0, x_cell_edges.size - 1):
        for yi in range(0, y_cell_edges.size - 1):
            for zi in range(0, z_cell_edges.size - 1):

                        # image block for histogram
                image_block = image_3D[
                                      x_cell_edges[xi]:x_cell_edges[xi+1],
                                      y_cell_edges[yi]:y_cell_edges[yi+1],
                                      z_cell_edges[zi]:z_cell_edges[zi+1]]

                        # histogram
                histogram_test[i, xi, yi, zi, :], bins = \
                            np.histogram(image_block,
                                         bins=np.linspace(0,
                                                          4500,
                                                          45 + 1))

X_test_new = np.reshape(histogram_test, (X_test.shape[0], -1))

In [79]:
X_train, X_val, y_train, y_val = train_test_split(X_new, y_train, test_size = 0.2)

In [80]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)

In [81]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

def custom_r2(preds, train_data):
    labels = train_data.get_label()
    return 'r2', roc_auc_score(labels, preds), True

print('Start training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=500,
                feval=custom_r2,
                valid_sets={lgb_train, lgb_eval},
                early_stopping_rounds=20)

Start training...
[1]	training's binary_logloss: 0.675043	training's r2: 0.813382	valid_0's binary_logloss: 0.690081	valid_0's r2: 0.566667
Training until validation scores don't improve for 20 rounds.
[2]	training's binary_logloss: 0.662861	training's r2: 0.873236	valid_0's binary_logloss: 0.6883	valid_0's r2: 0.539216
[3]	training's binary_logloss: 0.643978	training's r2: 0.927293	valid_0's binary_logloss: 0.689716	valid_0's r2: 0.564706
[4]	training's binary_logloss: 0.627704	training's r2: 0.927923	valid_0's binary_logloss: 0.689808	valid_0's r2: 0.578431
[5]	training's binary_logloss: 0.612605	training's r2: 0.948085	valid_0's binary_logloss: 0.695268	valid_0's r2: 0.509804
[6]	training's binary_logloss: 0.597785	training's r2: 0.957157	valid_0's binary_logloss: 0.689259	valid_0's r2: 0.556863
[7]	training's binary_logloss: 0.584658	training's r2: 0.959425	valid_0's binary_logloss: 0.690354	valid_0's r2: 0.533333
[8]	training's binary_logloss: 0.56708	training's r2: 0.962954	valid

In [82]:
y_pred = gbm.predict(X_test_new , num_iteration=gbm.best_iteration)

In [83]:
y_pred

array([0.25903254, 0.59793162, 0.65680447, 0.34189803, 0.78482993,
       0.30130879, 0.42660575, 0.77075514, 0.48404168, 0.39917007,
       0.72659622, 0.7021133 , 0.71013712, 0.15650399, 0.57215112,
       0.89444514, 0.3893225 , 0.54156065, 0.31918314, 0.50008797,
       0.63596871, 0.52178557, 0.29357271, 0.55076175, 0.74802448,
       0.26942731, 0.63659147, 0.81836689, 0.47553788, 0.71587711,
       0.74324246, 0.76630422, 0.31501518, 0.30960262, 0.626478  ,
       0.73396337, 0.3028767 , 0.57810541, 0.49543682, 0.54136336,
       0.81386465, 0.26456912, 0.39173781, 0.39678107, 0.50662276,
       0.36140976, 0.40656055, 0.47698813, 0.71849709, 0.75300414,
       0.83624386, 0.54306591, 0.27417854, 0.33596898, 0.71457679,
       0.10769183, 0.63705773, 0.53474473, 0.71813742, 0.72851383,
       0.24575929, 0.43223122, 0.67152991, 0.33443419, 0.58845098,
       0.51091206, 0.70624123, 0.45325788, 0.8018234 ])

In [84]:
f = open("submission.csv", "w")
f.write("id,y\n")
for i,x in enumerate(y_pred):
    f.write("{},{}\n".format(i,x))
f.close()

In [75]:
x_cell_number=4
y_cell_number=8
z_cell_number=8
theta_bin_number=18
phi_bin_number=9
IMAGE_DIM_X = 176
IMAGE_DIM_Y = 208
IMAGE_DIM_Z = 176
IMAGE_VALUE_MAX = 4500
IMAGE_FULL_FEATURE = 6443008

In [76]:
x_edges = np.linspace(0, 100,x_cell_number + 1,dtype=int)

y_edges = np.linspace(0,100,x_cell_number + 1,dtype=int)
z_edges = np.linspace(0,100,z_cell_number + 1,dtype=int)


histogram_train = np.zeros((X.shape[0],x_cell_number,y_cell_number,z_cell_number,theta_bin_number,phi_bin_number))
histogram_test = np.zeros((X_test.shape[0],x_cell_number,y_cell_number,z_cell_number,theta_bin_number,phi_bin_number))


In [77]:
for i in range(0, X.shape[0]):
    image_3D = X[i, :, :, :,0]
    gradient = np.gradient(image_3D.astype('float32'))

            # normalize
    norm = np.linalg.norm(gradient, axis=0)
    gradient = [np.where(norm == 0, 0, i / norm) for i in gradient]

    for xi in range(0, x_edges.size - 1):
        for yi in range(0, y_edges.size - 1):
            for zi in range(0, z_edges.size - 1):

                block_x = gradient[0][
                                  x_edges[xi]:x_edges[xi+1],
                                  y_edges[yi]:y_edges[yi+1],
                                  z_edges[zi]:z_edges[zi+1]]
                block_y = gradient[1][
                                  x_edges[xi]:x_edges[xi+1],
                                  y_edges[yi]:y_edges[yi+1],
                                  z_edges[zi]:z_edges[zi+1]]
                block_z = gradient[2][
                                  x_edges[xi]:x_edges[xi+1],
                                  y_edges[yi]:y_edges[yi+1],
                                  z_edges[zi]:z_edges[zi+1]]
                block_mag = norm[
                                    x_edges[xi]:x_edges[xi+1],
                                    y_edges[yi]:y_edges[yi+1],
                                    z_edges[zi]:z_edges[zi+1]]

                block_x = block_x.flatten()
                block_y = block_y.flatten()
                block_z = block_z.flatten()
                block_mag = block_mag.flatten()

                block_x = block_x[block_mag > 0.8]
                block_y = block_y[block_mag > 0.8]
                block_z = block_z[block_mag > 0.8]

                        # theta and phi
                theta_block = np.arctan2(block_y,
                                                 block_x) * 180.0 / np.pi
                phi_block = np.arccos(block_z) * 180.0 / np.pi

                bins = (np.linspace(-180, 180,
                                            theta_bin_number + 1),
                                np.linspace(0, 180,
                                            phi_bin_number + 1))

                        # histogram
                histogram_train[i, xi, yi, zi, :, :], theta_edge, phi_edge = \
                            np.histogram2d(theta_block, phi_block, bins=bins)

    X_new = np.reshape(histogram_train, (X.shape[0], -1))
    

/anaconda3/lib/python3.6/site-packages/numpy/linalg/linalg.py:2390: RuntimeWarning: invalid value encountered in sqrt
  return sqrt(add.reduce(s, axis=axis, keepdims=keepdims))
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


In [78]:
for i in range(0, X_test.shape[0]):
    image_3D = X_test[i, :, :, :,0]
    gradient = np.gradient(image_3D.astype('float32'))

            # normalize
    norm = np.linalg.norm(gradient, axis=0)
    gradient = [np.where(norm == 0, 0, i / norm) for i in gradient]

    for xi in range(0, x_edges.size - 1):
        for yi in range(0, y_edges.size - 1):
            for zi in range(0, z_edges.size - 1):

                block_x = gradient[0][
                                  x_edges[xi]:x_edges[xi+1],
                                  y_edges[yi]:y_edges[yi+1],
                                  z_edges[zi]:z_edges[zi+1]]
                block_y = gradient[1][
                                  x_edges[xi]:x_edges[xi+1],
                                  y_edges[yi]:y_edges[yi+1],
                                  z_edges[zi]:z_edges[zi+1]]
                block_z = gradient[2][
                                  x_edges[xi]:x_edges[xi+1],
                                  y_edges[yi]:y_edges[yi+1],
                                  z_edges[zi]:z_edges[zi+1]]
                block_mag = norm[
                                    x_edges[xi]:x_edges[xi+1],
                                    y_edges[yi]:y_edges[yi+1],
                                    z_edges[zi]:z_edges[zi+1]]

                block_x = block_x.flatten()
                block_y = block_y.flatten()
                block_z = block_z.flatten()
                block_mag = block_mag.flatten()

                block_x = block_x[block_mag > 0.8]
                block_y = block_y[block_mag > 0.8]
                block_z = block_z[block_mag > 0.8]

                        # theta and phi
                theta_block = np.arctan2(block_y,
                                                 block_x) * 180.0 / np.pi
                phi_block = np.arccos(block_z) * 180.0 / np.pi

                bins = (np.linspace(-180, 180,
                                            theta_bin_number + 1),
                                np.linspace(0, 180,
                                            phi_bin_number + 1))

                        # histogram
                histogram_test[i, xi, yi, zi, :, :], theta_edge, phi_edge = \
                            np.histogram2d(theta_block, phi_block, bins=bins)

    X_test_new = np.reshape(histogram_test, (X_test.shape[0], -1))

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_new, y_train, test_size = 0.2)

In [ ]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)

In [ ]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

def custom_r2(preds, train_data):
    labels = train_data.get_label()
    return 'r2', roc_auc_score(labels, preds), True

print('Start training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=500,
                feval=custom_r2,
                valid_sets={lgb_train, lgb_eval},
                early_stopping_rounds=20)

In [ ]:
y_pred = gbm.predict(X_test_new , num_iteration=gbm.best_iteration)

In [ ]:
f = open("submission.csv", "w")
f.write("id,y\n")
for i,x in enumerate(y_pred):
    f.write("{},{}\n".format(i,x))
f.close()